**問題1**：為什麼要使用 Chinese\_Taiwan\_Stroke\_CI\_AS 定序與 Unicode 支援

<span class="s1" style="color: rgb(0, 0, 0);">參考：</span>[https://docs.microsoft.com/zh-tw/sql/relational-databases/collations/collation-and-unicode-support?view=sql-server-ver15](https://docs.microsoft.com/zh-tw/sql/relational-databases/collations/collation-and-unicode-support?view=sql-server-ver15)

Chinese-Taiwan-Stroke、排序不區分大小寫、區分腔調

<span class="s2" style="font-variant-numeric: normal; font-variant-east-asian: normal; font-stretch: normal; line-height: normal; font-family: &quot;PingFang TC&quot;;">同時在使用</span>  NCHAR<span class="s2" style="font-variant-numeric: normal; font-variant-east-asian: normal; font-stretch: normal; line-height: normal; font-family: &quot;PingFang TC&quot;;">、</span>NVARCHAR <span class="s2" style="font-variant-numeric: normal; font-variant-east-asian: normal; font-stretch: normal; line-height: normal; font-family: &quot;PingFang TC&quot;;">和</span> NTEXT <span class="s2" style="font-variant-numeric: normal; font-variant-east-asian: normal; font-stretch: normal; line-height: normal; font-family: &quot;PingFang TC&quot;;">資料類型時能支援</span>  unicode <span class="s2" style="font-variant-numeric: normal; font-variant-east-asian: normal; font-stretch: normal; line-height: normal; font-family: &quot;PingFang TC&quot;;">及</span> emoji

In [ ]:
-- Warnng:建立資料庫 -- 
CREATE DATABASE [shop]
    COLLATE Chinese_Taiwan_Stroke_CI_AS;  
GO

In [ ]:
-- 切換使用資料庫 --
USE [shop];
GO

**PK** - **主鍵**

**FK - 外部鍵**

**IX** - 索引(非唯一值)

**AK** - 索引(唯一值)

In [ ]:
-- Warning：刪除[會員]資料表 --
DROP TABLE IF EXISTS [dbo].[Members]
GO

-- 建立[會員]資料表 --
CREATE TABLE [dbo].[Members] (
    [ID]            BIGINT         IDENTITY (1, 1) NOT NULL,
    [Login]         VARCHAR (60)   NOT NULL,
    [Pass]          VARCHAR (255)  DEFAULT ('') NOT NULL,
    [Nicename]      NVARCHAR (50)  DEFAULT ('') NOT NULL,
    [DisplayName]   NVARCHAR (250) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [Email]         VARCHAR (100)  DEFAULT ('') NOT NULL,
    [Url]           VARCHAR (100)  DEFAULT ('') NOT NULL,
    [Created]       DATETIME       DEFAULT (getdate()) NOT NULL,
    [Modified]      DATETIME       DEFAULT (getdate()) NOT NULL,
    [ActivationKey] VARCHAR (255)  DEFAULT ('') NOT NULL,
    [IsStaff]       BIT            DEFAULT ((0)) NOT NULL,
    [Status]        TINYINT        DEFAULT ((0)) NOT NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC)
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_Login]
    ON [dbo].[Members]([Login] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Nicename]
    ON [dbo].[Members]([Nicename] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Email]
    ON [dbo].[Members]([Email] ASC);
GO

**問題2**：為什麼不該使用 NTEXT？

因為 NTEXT 是大型物件(LOB)結構，這類的資料類型可以存放到 2GB 的資料量，

而 NVARCHAR(MAX) 預設在不超過 8,000 Bytes 的情況下，資料是儲存在資料表裡的，只有超過時才改存到 LOB 裡。  
改用 NVARCHAR(MAX) 在儲存空間、查詢、交易複寫等，相對會比較有優勢。

另外撰寫 .NET 程式時也相較容易。

會員級別：member\_grade  

信用評比：credit\_rating

  

**使用 JSON 格式**  
單一登入：LINE ID,  Facebook ID, Gmail  

信用卡 credit\_cards

地址簿 address\_book (出貨、帳單地址) 

發票簿 invoice\_book (二聯、三聯、統一編號、公司抬頭)

In [ ]:
-- Warning：刪除[會員詮釋資料]資料表 --
DROP TABLE IF EXISTS [dbo].[MemberMeta]
GO

-- 建立[會員詮釋資料]資料表 --
CREATE TABLE [dbo].[MemberMeta] (
    [MMetaID]   BIGINT          IDENTITY (1, 1) NOT NULL,
    [MemberID]  BIGINT             NOT NULL,
    [MetaKey]   VARCHAR (255)   DEFAULT ('') NULL,
    [MetaValue] NVARCHAR (MAX)  DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NULL,
    PRIMARY KEY CLUSTERED ([MMetaID] ASC)
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_MemberID]
    ON [dbo].[MemberMeta]([MemberID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_MemberMetaKey]
    ON [dbo].[MemberMeta]([MetaKey] ASC);
GO

In [ ]:
-- Warning：刪除[商品]資料表 --
DROP TABLE IF EXISTS [dbo].[Products]
GO

-- 建立[商品]資料表 --
CREATE TABLE [dbo].[Products] (
    [ID]                     BIGINT         IDENTITY (1, 1) NOT NULL,
    [ProductAuthor]          BIGINT         DEFAULT (0) NOT NULL,
    [ProductContent]         NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [ProductTitle]           NVARCHAR (200) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [ProductExcerpt]         NVARCHAR (1000) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [ProductStatus]          VARCHAR (20)   DEFAULT ('publish') NOT NULL,
    [ProductPassword]        VARCHAR (255)  DEFAULT ('') NOT NULL,
    [ProductName]            VARCHAR (200)  DEFAULT ('') NOT NULL,
    [Ping]                   NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [PingStatus]             VARCHAR (20)   DEFAULT ('open') NOT NULL,
    [Follow]                 NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [FollowStatus]           VARCHAR (20)   DEFAULT ('active') NOT NULL,
    [Created]            DATETIME       DEFAULT (getdate()) NOT NULL,
    [Modified]        DATETIME       DEFAULT (getdate()) NOT NULL,
    [ProductContentFiltered] NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [ProductParent]          BIGINT         DEFAULT (0) NOT NULL,
    [Guid]                   VARCHAR (1000)  DEFAULT ('') NOT NULL,
    [MenuOrder]              INT            DEFAULT (0) NOT NULL,
    [ProductType]            VARCHAR (20)   DEFAULT ('physical') NOT NULL,
    [ProductMimeType]        VARCHAR (100)  DEFAULT ('') NOT NULL,
    [CommentStatus]          VARCHAR (20)   DEFAULT ('open') NOT NULL,
    [CommentCount]           BIGINT         DEFAULT (0) NOT NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC)
);
GO

CREATE NONCLUSTERED INDEX [ix_ProductParent]
    ON [dbo].[Products]([ProductParent] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_ProductName]
    ON [dbo].[Products]([ProductName] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_ProductAuthor]
    ON [dbo].[Products]([ProductAuthor] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_TypeStatusDate]
    ON [dbo].[Products]([ProductType] ASC, [ProductStatus] ASC, [Created] ASC, [ID] ASC);
GO

**商品屬性**

庫存單位：sku

虛擬商品：virtual

可否下載：downloadable

最低價格：min\_price

最高價格：max\_price

銷售中：onsale

庫存數量：stock\_quantity

庫存狀態：stock\_status

評分次數：rating\_count

評分平均：average\_rating

已銷售數量：total\_sales

稅狀態： tax\_status

稅類： tax\_class

In [ ]:
-- Warning：刪除[商品詮釋資料]資料表 --
DROP TABLE IF EXISTS [dbo].[ProductMeta]
GO

-- 建立[商品詮釋資料]資料表 --
CREATE TABLE [dbo].[ProductMeta] (
    [PMetaID]   BIGINT         IDENTITY (1, 1) NOT NULL,
    [ProductID] BIGINT         NOT NULL,
    [MetaKey]   VARCHAR (255)  DEFAULT ('') NULL,
    [MetaValue] NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NULL,
    PRIMARY KEY CLUSTERED ([PMetaID] ASC)
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_ProductID]
    ON [dbo].[ProductMeta]([ProductID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_ProductMetaKey]
    ON [dbo].[ProductMeta]([MetaKey] ASC);
GO

**公司類型**

通路商 channel

代理商 agent

零售 retailer

品牌 brand

製造 manufacture

網路商店 shop

In [ ]:
-- 因為公司主鍵 ID 同時繫結了商店，作為其外部索引鍵，所以要刪除公司資料表前，需先刪除此外部索引 -- 
ALTER TABLE [dbo].[Stores]
DROP CONSTRAINT [fk_CompanyID];
GO

-- Warning：刪除[公司]資料表 --
/*
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[Companies]') AND type in (N'U'))
DROP TABLE [dbo].[Companies]
GO
*/
DROP TABLE IF EXISTS [dbo].[Companies]
GO

-- 建立[公司]資料表 --
CREATE TABLE [dbo].[Companies] (
    [ID]                BIGINT         IDENTITY (1, 1) NOT NULL,
    [CompanyName]       NVARCHAR (100) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [CompanySupervisor] BIGINT         DEFAULT (0) NOT NULL,
    [CompanyDesc]       NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [CompanyType]       VARCHAR (20)   DEFAULT ('retailer') NOT NULL,
    [CountryCode]       VARCHAR (10)   DEFAULT ('') NOT NULL,
    [TaxID]             VARCHAR (20)   DEFAULT ('') NOT NULL,
    [Url]               VARCHAR (100)  DEFAULT ('') NOT NULL,
    [Email]             VARCHAR (100)  DEFAULT ('') NOT NULL,
    [Phone]             VARCHAR (100)  DEFAULT ('') NOT NULL,
    [Created]         DATETIME       DEFAULT (getdate()) NOT NULL,
    [Modified]   DATETIME       DEFAULT (getdate()) NOT NULL,
    [Status]            TINYINT        DEFAULT ((0)) NOT NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC)
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_CompanyName]
    ON [dbo].[Companies]([CompanyName] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_CountryCode]
    ON [dbo].[Companies]([CountryCode] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_TaxID]
    ON [dbo].[Companies]([TaxID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Url]
    ON [dbo].[Companies]([Url] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Email]
    ON [dbo].[Companies]([Email] ASC);
GO

In [ ]:
-- Warning：刪除[公司詮釋資料]資料表 --
DROP TABLE IF EXISTS [dbo].[CompanyMeta]
GO

-- 建立[公司詮釋資料]資料表 --
CREATE TABLE [dbo].[CompanyMeta] (
    [CMetaID]   BIGINT         IDENTITY (1, 1) NOT NULL,
    [CompanyID] BIGINT         NOT NULL,
    [MetaKey]   VARCHAR (255)  DEFAULT ('') NULL,
    [MetaValue] NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NULL,
    PRIMARY KEY CLUSTERED ([CMetaID] ASC)
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_CompanyID]
    ON [dbo].[CompanyMeta]([CompanyID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_CompanyMetaKey]
    ON [dbo].[CompanyMeta]([MetaKey] ASC);
GO

In [ ]:
-- Warning：刪除[商店]資料表 --
DROP TABLE IF EXISTS [dbo].[Stores]
GO

-- 建立[商店]資料表 --
CREATE TABLE [dbo].[Stores] (
    [ID]              BIGINT         IDENTITY (1, 1) NOT NULL,
    [CompanyID]       BIGINT         DEFAULT (0) NOT NULL,
    [StoreName]       NVARCHAR (100) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [StoreSupervisor] BIGINT         DEFAULT (0) NOT NULL,
    [StoreDesc]       NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [Phone]           VARCHAR (100)  DEFAULT ('') NOT NULL,
    [CountryCode]     VARCHAR (10)   DEFAULT ('') NOT NULL,
    [ZipCode]         VARCHAR (10)   DEFAULT ('') NOT NULL,
    [State]           NVARCHAR (30)  DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [City]            NVARCHAR (40)  DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [Address]         NVARCHAR (200) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [Created]       DATETIME       DEFAULT (getdate()) NOT NULL,
    [Modified]   DATETIME       DEFAULT (getdate()) NOT NULL,
    [Status]          TINYINT        DEFAULT ((0)) NOT NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC),
    CONSTRAINT [fk_CompanyID] FOREIGN KEY ([CompanyID]) REFERENCES [dbo].[Companies] ([ID])
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_Phone]
    ON [dbo].[Stores]([Phone] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_CountryCode]
    ON [dbo].[Stores]([CountryCode] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_ZipCode]
    ON [dbo].[Stores]([ZipCode] ASC);
GO

In [ ]:
-- Warning：刪除[商店雇員](權限)資料表 --
DROP TABLE IF EXISTS [dbo].[StoreStaff]
GO

-- 建立[商店雇員](權限)資料表 --
CREATE TABLE [dbo].[StoreStaff] (
    [ID]              BIGINT         IDENTITY (1, 1) NOT NULL,
    [StoreID]       BIGINT         DEFAULT (0) NOT NULL,
    [StaffID]       BIGINT         DEFAULT (0) NOT NULL,
    [Permission]    NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [Created]       DATETIME       DEFAULT (getdate()) NOT NULL,
    [Modified]   DATETIME       DEFAULT (getdate()) NOT NULL,
    [Status]          TINYINT        DEFAULT ((0)) NOT NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC),
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_StoreID]
    ON [dbo].[StoreStaff]([StoreID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Created]
    ON [dbo].[StoreStaff]([Created] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_StaffID]
    ON [dbo].[StoreStaff]([StaffID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Status]
    ON [dbo].[StoreStaff]([Status] ASC);
GO

In [ ]:
-- Warning：刪除[商店庫存]資料表 --
DROP TABLE IF EXISTS [dbo].[Stocks]
GO

-- 建立[商店庫存]資料表 --
CREATE TABLE [dbo].[Stocks] (
    [ID]              BIGINT         IDENTITY (1, 1) NOT NULL,
    [StoreID]         BIGINT         DEFAULT (0) NOT NULL,
    [ProductID]       BIGINT         DEFAULT (0) NOT NULL,
    [Quantity]        DECIMAL(10,2)  DEFAULT (0) NOT NULL,
    [Created]         DATETIME       DEFAULT (getdate()) NOT NULL,
    [Modified]        DATETIME       DEFAULT (getdate()) NOT NULL,
    [Status]          TINYINT        DEFAULT ((0)) NOT NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC),
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_StoreID]
    ON [dbo].[Stocks]([StoreID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Created]
    ON [dbo].[Stocks]([Created] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_ProductID]
    ON [dbo].[Stocks]([ProductID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_ProductQuantity]
    ON [dbo].[Stocks](ProductID, [Quantity] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Status]
    ON [dbo].[Stocks]([Status] ASC);
GO

In [ ]:
-- Warning：刪除[分類關聯]資料表 --
DROP TABLE IF EXISTS [dbo].[TermRelationships]
GO

-- 建立[分類關聯]資料表 --
CREATE TABLE [dbo].[TermRelationships] (
    [ObjectID]       BIGINT         DEFAULT (0) NOT NULL,
    [TermTaxonomyID]  BIGINT         DEFAULT (0) NOT NULL,
    [TermOrder]       INT            DEFAULT (0) NOT NULL,
    PRIMARY KEY CLUSTERED ([ObjectID], [TermTaxonomyID]),
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_TermTaxonomyID]
    ON [dbo].[TermRelationships]([TermTaxonomyID] ASC);
GO

In [ ]:
-- Warning：刪除[分類法及項目]資料表 --
DROP TABLE IF EXISTS [dbo].[TermTaxonomy]
GO

-- 建立[分類法及項目]資料表 --
CREATE TABLE [dbo].[TermTaxonomy] (
    [TermTaxonomyID]  BIGINT         IDENTITY (1, 1) NOT NULL,
    [TermID]          BIGINT         DEFAULT (0) NOT NULL,
    [Taxonomy]        VARCHAR (20)   DEFAULT ('') NOT NULL,
    [Description]     NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [Parent]          BIGINT         DEFAULT (0) NOT NULL,
    [Count]           BIGINT         DEFAULT (0) NOT NULL,
    PRIMARY KEY CLUSTERED ([TermTaxonomyID]),
    CONSTRAINT ak_TermIDTaxonomy UNIQUE([TermID], [Taxonomy]) 
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_Taxonomy]
    ON [dbo].[TermTaxonomy]([Taxonomy] ASC);
GO

In [ ]:
-- Warning：刪除[分類項目]資料表 --
DROP TABLE IF EXISTS [dbo].[Terms]
GO

-- 建立[分類項目]資料表 --
CREATE TABLE [dbo].[Terms] (
    [TermID]    BIGINT           IDENTITY (1, 1) NOT NULL,
    [Name]      NVARCHAR (200)   DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [Slug]      VARCHAR (1000)   DEFAULT ('') NOT NULL,
    [TermGroup] BIGINT           DEFAULT (0) NOT NULL,
    PRIMARY KEY CLUSTERED ([TermID])
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_Name]
    ON [dbo].[Terms]([Name] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Slug]
    ON [dbo].[Terms]([Slug] ASC);
GO

In [ ]:
-- Warning：刪除[分類詮釋資料]資料表 --
DROP TABLE IF EXISTS [dbo].[TermMeta]
GO

-- 建立[分類詮釋資料]資料表 --
CREATE TABLE [dbo].[TermMeta] (
    [TMetaID]   BIGINT         IDENTITY (1, 1) NOT NULL,
    [TermID]    BIGINT         DEFAULT (0) NOT NULL,
    [MetaKey]   VARCHAR (255)  DEFAULT ('') NULL,
    [MetaValue] NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NULL,
    PRIMARY KEY CLUSTERED ([TMetaID])
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_TermID]
    ON [dbo].[TermMeta]([TermID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_MetaKey]
    ON [dbo].[TermMeta]([MetaKey] ASC);
GO

In [ ]:
-- Warning：刪除[訂單主表及狀態]資料表 --
DROP TABLE IF EXISTS [dbo].[OrderStats]
GO

-- 建立[訂單主表及狀態]資料表 --
CREATE TABLE [dbo].[OrderStats] (
    [OrderID]              BIGINT         IDENTITY (1, 1) NOT NULL,
    [CustomerID]           BIGINT         DEFAULT (0) NOT NULL,
    [ParentID]             BIGINT         DEFAULT (0) NOT NULL,
    [NumItemsSold]         INT         DEFAULT (0) NOT NULL,
    [TotalSales]           DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [TaxTotal]             DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [ShippingTotal]        DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [NetTotal]             DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [ReturningCustomer]    TINYINT DEFAULT NULL,
    [Created]              DATETIME       DEFAULT (getdate()) NOT NULL,
    [Status]               VARCHAR(200)        DEFAULT ('') NOT NULL,
    PRIMARY KEY CLUSTERED ([OrderID] ASC),
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_CustomerID]
    ON [dbo].[OrderStats]([CustomerID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Created]
    ON [dbo].[OrderStats]([Created] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Status]
    ON [dbo].[OrderStats]([Status] ASC);
GO

In [ ]:
-- Warning：刪除[訂單商品項目]資料表 --
DROP TABLE IF EXISTS [dbo].[OrderProduct]
GO

-- 建立[訂單商品項目]資料表 --
CREATE TABLE [dbo].[OrderProduct] (
    [OrderItemID]              BIGINT         IDENTITY (1, 1) NOT NULL,
    [OrderID]                  BIGINT         DEFAULT (0) NOT NULL,
    [ProductID]                BIGINT         DEFAULT (0) NOT NULL,
    [VariationID]              BIGINT         DEFAULT (0) NOT NULL,
    [CustomerID]               BIGINT         DEFAULT NULL,
    [ProductQuantity]          DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [ProductNetRevenue]        DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [ProductGrossRevenue]      DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [CouponAmount]             DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [TaxAmount]                DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [ShippingAmount]           DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [ShippingTaxAmount]        DECIMAL(10,2)         DEFAULT (0) NOT NULL,
    [Created]                  DATETIME       DEFAULT (getdate()) NOT NULL,
    PRIMARY KEY CLUSTERED ([OrderItemID] ASC),
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_OrderID]
    ON [dbo].[OrderProduct]([OrderID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_ProductID]
    ON [dbo].[OrderProduct]([ProductID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_CustomerID]
    ON [dbo].[OrderProduct]([CustomerID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Created]
    ON [dbo].[OrderProduct]([Created] ASC);
GO

出貨地址 Shipping Address

帳單地址 Billing Address

發票編號 Invoice ID

In [ ]:
-- Warning：刪除[訂單詮釋資料]資料表 --
DROP TABLE IF EXISTS [dbo].[OrderMeta]
GO

-- 建立[訂單詮釋資料]資料表 --
CREATE TABLE [dbo].[OrderMeta] (
    [OMetaID]   BIGINT         IDENTITY (1, 1) NOT NULL,
    [OrderID]    BIGINT         DEFAULT (0) NOT NULL,
    [MetaKey]   VARCHAR (255)  DEFAULT ('') NULL,
    [MetaValue] NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NULL,
    PRIMARY KEY CLUSTERED ([OMetaID])
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_OrderID]
    ON [dbo].[OrderMeta]([OrderID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_MetaKey]
    ON [dbo].[OrderMeta]([MetaKey] ASC);
GO

In [ ]:
-- Warning：刪除[訂單使用折價券]資料表 --
DROP TABLE IF EXISTS [dbo].[OrderCoupon]
GO

-- 建立[訂單使用折價券]資料表 --
CREATE TABLE [dbo].[OrderCoupon] (
    [OrderID]              BIGINT         DEFAULT (0) NOT NULL,
    [CouponID]             BIGINT         DEFAULT (0) NOT NULL,
    [DiscountAmount]       DECIMAL(10,2)  DEFAULT (0) NOT NULL,
    [Created]              DATETIME       DEFAULT (getdate()) NOT NULL,
    PRIMARY KEY CLUSTERED ([OrderID], [CouponID]),
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_CouponID]
    ON [dbo].[OrderCoupon]([CouponID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Created]
    ON [dbo].[OrderCoupon]([Created] ASC);
GO

In [ ]:
-- Warning：刪除[積分獎勵]資料表 --
DROP TABLE IF EXISTS [dbo].[PointReward]
GO

-- 建立[積分獎勵]資料表 --
CREATE TABLE [dbo].[PointReward] (
    [ID]              BIGINT         IDENTITY (1, 1) NOT NULL,
    [CustomerID]      BIGINT         DEFAULT (0) NOT NULL,
    [OrderID]         BIGINT         DEFAULT (0) NOT NULL,
    [Points]          BIGINT         DEFAULT (0) NOT NULL,
    [PointsBalance]   BIGINT         DEFAULT (0) NOT NULL,
    [Created]         DATETIME       DEFAULT (getdate()) NOT NULL,
    PRIMARY KEY CLUSTERED ([ID]),
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_UserPointsBalance]
    ON [dbo].[PointReward]([CustomerID], [PointsBalance]);
GO

CREATE NONCLUSTERED INDEX [ix_DatePointsBalance]
    ON [dbo].[PointReward]([Created], [PointsBalance]);
GO